In [ ]:
import pandas as pd
import os
from alchemlyb.parsing import amber
from alchemlyb.estimators import TI,MBAR

In [ ]:
# repate number
round_NO = 1
# restraint constant
k_dist=10
k_ang=100
k_rotate=100

In [ ]:
top_dir = os.getcwd()
os.chdir(f'./complex_{round_NO}_{k_dist}_{k_ang}_{k_rotate}/ligand_one_step')
main_dir = os.getcwd()
file_list = [0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

In [ ]:
dHdl = pd.concat([amber.extract_dHdl(str(filename) + '/ti001.out', T=300) for filename in file_list])
u_nk_rm_rest = pd.concat([amber.extract_u_nk(str(filename) + '/ti001.out', T=300) for filename in file_list])

In [ ]:
ti_estimator = TI().fit(dHdl)
mbar_estimator = MBAR().fit(u_nk_rm_rest)

In [ ]:
delta_f = ti_estimator.delta_f_.iloc[0, -1]
d_delta_f = ti_estimator.d_delta_f_.iloc[0, -1]

In [ ]:
print("TI:")
print("Free energy difference (TI)", delta_f, "kT")
print("Free energy difference (TI) error", d_delta_f, "kT")

In [ ]:
print("Free energy difference (MBAR)", mbar_estimator.delta_f_.loc[0.0, 1.0],'kT')
from alchemlyb.visualisation import plot_ti_dhdl,plot_mbar_overlap_matrix
ax = plot_ti_dhdl([ti_estimator], labels=['dHdl'], colors=['r'],units='kcal/mol')
ax.figure.savefig('onestep_dhdl_TI.pdf')

In [ ]:
ax = plot_mbar_overlap_matrix(mbar_estimator.overlap_matrix)
ax.figure.savefig('onestep_MBAR.pdf', bbox_inches='tight', pad_inches=0.0)

In [ ]:
import matplotlib.pyplot as plt
from alchemlyb.visualisation.dF_state import plot_dF_state

In [ ]:
estimators = [(ti_estimator), (mbar_estimator)]

fig = plot_dF_state(estimators, orientation='portrait',units='kcal/mol')
fig.savefig('dF_state.pdf', bbox_inches='tight')

with open(f'{main_dir}/result.txt','w') as f:
    f.write(f'Free energy difference (TI):\t{delta_f * 0.596:0.3f} ± {d_delta_f * 0.596:0.3f} kcal/mol\n')
    f.write(f'Free energy difference (MBAR):\t{mbar_estimator.delta_f_.loc[0.0,1.0] * 0.596:0.3f} ± {mbar_estimator.d_delta_f_.loc[0.0,1.0] * 0.596:0.3f} kcal/mol\n')
    for i in range(len(file_list)-1):
        f.writelines(f'{mbar_estimator.delta_f_.loc[file_list[i],file_list[i+1]] * 0.596:.3f}\n')

with open(f'{top_dir}/summary_ABFE.txt','a') as f:
    f.write('ligand decouple from solvent\n')
    f.write(f'\tTi: {delta_f * 0.596:0.3f} ± {d_delta_f * 0.596:0.3f}\t MBAR: {mbar_estimator.delta_f_.loc[0.0,1.0] * 0.596:0.3f} ± {mbar_estimator.d_delta_f_.loc[0.0,1.0] * 0.596:0.3f} kcal/mol\n')

In [ ]:
delta_f

In [ ]:
type(ti_estimator.delta_f_.iloc[0, -1])

In [ ]:
ti_estimator.delta_f_

In [ ]:
mbar_estimator.delta_f_

In [ ]:
mbar_estimator.delta_f_.iloc[0,-1] * 0.6